#목차
이번 실습은 <b>1) 네이버 감성분석 데이터 불러오기 및 전처리 2) BERT 인풋 만들기 3) 버트를 활용한 감성분석 모델 만들기 4) 훈련 및 성능 검증 5) 실제 데이터로 실습하기</b>로 구성되어 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 분석 데이터 불러오기 및 전처리


huggingface 패키지를 Colab에 설치합니다

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.2MB/s 
     |████████████████████████████████| 2.9MB 18.3MB/s 
     |████████████████████████████████| 890kB 34.8MB/s 
     |████████████████████████████████| 1.1MB 33.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bba23300a228837afbdca5cdffc1ecaaed5333fe5fbcc09e0ae7a097095c6421
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


텐서플로우 2와 필요한 모듈들을 임포트합니다.  
최근에 텐서플로우 기본 버전은 2로 바뀌었습니다.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

이번 예제에서 사용할 네이버 영화 감상분석 데이터를 다운로드 합니다

In [ ]:
# csv 읽어오기
col_names=['label', 'document']
best_train = pd.read_csv("/content/drive/MyDrive/감정분석_avengers/best_train.csv", names= col_names)
worst_train = pd.read_csv("/content/drive/MyDrive/감정분석_avengers/worst_train.csv", names= col_names)
test = pd.read_csv("/content/drive/MyDrive/감정분석_avengers/test.csv",names=col_names)

In [ ]:
train = pd.concat([best_train,worst_train])

In [ ]:
train = train.dropna()

In [ ]:
train = train[['document','label']]

In [ ]:
train = train.dropna().reset_index()
train = train[['document','label']]
train

,document,label
0,스포일러가 포함된 감상평입니다. 감상평 보기아이언맨으로 시작해서 아이언맨으로 끝나는...,1
1,첨볼땐 와~하고봤는데 두번볼때 울었네ㅜㅜ,1
2,마블펜이라면 무조건 봐야하는 영화... 진짜 처음에 아이언맨1편 나오고 토르1편나올...,1
3,스포일러가 포함된 감상평입니다. 감상평 보기지금은 늣었지만 아이언맨 존경합니다,1
4,이제 영화 뭐 보면서 살지ㅠㅠ,1
...,...,...
7735,관람객그냥 괜찮네용괜찮아ㅇㅇ,0
7736,관람객2시간 넘는 시간동안 영화가좀 지루해서 몸을 배배꼬꼬 여러차레자세를 바꾸고 하...,0
7737,관람객마블팬도 뭣도 아닌 입장에서 순수하게 봤을때 스토리 진행도 진부하고 액션이나 ...,0
7738,"볼만한데 전 작품들보다 연출, 편집, 오락성등등 모든게 부족했던것같아요~ 너무 기대...",0


In [ ]:
test[test['label']==0]

,label,document
420,0,생각보다 기대가 커서 그런지 지루하게봄..내3시간..
421,0,리얼 인생영화네요 3000만큼 사랑합니다
422,0,캐릭터 벨붕때매 2점 깐다
423,0,마지막이라 다들 모였지만 기대에는 못 미쳤다. 감동과 오글거림의 아슬아슬한 줄타기....
424,0,마블 사랑해요 999점 드립니다
...,...,...
9845,0,관람객또보고싶은 최고의 영화
9846,0,미쳤습니다 진짜로지루할 틈이 없습니다
9847,0,3시간이라서 다행.. 주요 캐릭터의 감정씬들도 꼭 필요했다고 봄. 오글거리는 씬이나...
9848,0,관람객인피니티스톤 없는 타노스가 더 쎄다


In [ ]:
test = test[['document','label']]

In [ ]:
lenth = []
for i in train.document:
    lenth.append(len(i))
    

In [ ]:
max(lenth)

147

In [ ]:
print(train.label.value_counts())
print ("="*26)
print(test.label.value_counts())

0    3998
1    3742
Name: label, dtype: int64
1    9530
0     470
Name: label, dtype: int64


## 버트 인풋 만들기

한글 데이터를 분석하려면, 100개가 넘는 언어에 대해 훈련된 버트를 사용해야 합니다.  
multilingual BERT를 사용하도록 하겠습니다.  
모델을 로드하기에 앞서, 토크나이저를 불러오도록 하겠습니다.  
huggingface에서는 아주 쉽게 토크나이저를 불러올 수 있습니다.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

버트를 사용하기에 앞서 가장 기초에 속하는 tokenizer 사용 방법에 대해서 잠시 배워보도록 하겠습니다.  
tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌  
tokenizer.tokenize => 문장을 토큰화

In [ ]:
# context vector  연습


In [ ]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 147 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 147
BATCH_SIZE = 20
# 설명변수 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 자살 생각 여부 ( 0: 없음, 1: 있음 )를 포함하고 있는 칼럼
LABEL_COLUMN = "label"



In [ ]:
# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

100%|██████████| 7740/7740 [00:02<00:00, 2902.08it/s]


In [ ]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 10000/10000 [00:03<00:00, 3056.16it/s]


## 버트를 활용한 감성분석 모델 만들기

버트 훈련을 빠르게 하기 위해, TPU를 사용하도록 하겠습니다.  
TPU를 사용하시고 싶지 않으신 분은 그냥 TPU 관련 부분을 실행하지 않으면 되겠습니다.

In [ ]:
# TPU 객체 지정
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

INFO:tensorflow:Initializing the TPU system: grpc://10.47.41.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.47.41.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

bert_outputs = bert_outputs[1]
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
sentiment_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 147)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 147)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 147)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   ((None, 147, 768), ( 177853440   input_word_ids[0][0]             
                                                                 input_masks[0][0]     

감정 분석에 맞는 버트 모형을 리턴하는 함수를 정의하도록 하겠습니다.  
TPU를 활용하려면 함수로 묶어야 합니다.

In [ ]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=2.0e-5, weight_decay=0.0025)


sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])

In [ ]:
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])
  return sentiment_model

## 훈련 및 성능 검증

In [ ]:
# TPU 실행 시
if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
# 함수를 strategy.scope로 묶어 줌
  with strategy.scope():
    sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=3, shuffle=True, batch_size=25, validation_data=(test_x, test_y))
else:
  # GPU 모드로 훈련시킬 때
  sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=3, shuffle=True, batch_size=25, validation_data=(test_x, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
  2/310 [..............................] - ETA: 1:41:39 - loss: 0.6734 - acc: 0.6400WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0091s vs `on_train_batch_end` time: 0.1143s). Check your callbacks.


310/310 [==============================] - ETA: 0s - loss: 0.5504 - acc: 0.7137WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_test_batch_end` time: 0.0309s). Check your callbacks.


310/310 [==============================] - 139s 450ms/step - loss: 0.5504 - acc: 0.7137 - val_loss: 0.6004 - val_acc: 0.7345
Epoch 2/3
310/310 [==============================] - 52s 167ms/step - loss: 0.3842 - acc: 0.8313 - val_loss: 0.6440 - val_acc: 0.7397
Epoch 3/3
310/310 [==============================] - 52s 167ms/step - loss: 0.3085 - acc: 0.8711 - val_loss: 0.4633 - val_acc: 0.8177


훈련한 모델을 path에 저장
path는 임의로 지정해 주세요

In [ ]:
# PATH는 임의로 지정

#path = "/content/drive/MyDrive/한국어bert (pretrainedmodel)"

In [ ]:
#sentiment_model.save_weights(path+"/kyrbs_sui_con_upsample.h5")

Training된 가중치 들고와서 테스트할 때

In [ ]:
#'''sentiment_model = create_sentiment_bert()
#sentiment_model.load_weights(path + "/kyrbs_sui_con_upsample.h5")

훈련 모델의 예측 성능을 F1 SCORE로 체크하기 위한 작업

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [ ]:
test_set = predict_load_data(test)

100%|██████████| 10000/10000 [00:03<00:00, 3093.93it/s]


In [ ]:
test_set

[array([[  101,  8915,  8915, ...,     0,     0,     0],
        [  101,  9640, 97146, ...,     0,     0,     0],
        [  101,  9670, 89523, ...,     0,     0,     0],
        ...,
        [  101,  8900, 61250, ...,     0,     0,     0],
        [  101,  8915, 42144, ...,     0,     0,     0],
        [  101,  8900, 61250, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
#test set 가져오기
#np.save('/content/drive/MyDrive/한국어bert (pretrainedmodel)/test_set', test_set) # x_save.npy
#test_set = np.load('/content/drive/MyDrive/한국어bert (pretrainedmodel)/test_set.npy')

In [ ]:
preds = sentiment_model.predict(test_set)
preds

array([[0.9626856 ],
       [0.16960557],
       [0.94798195],
       ...,
       [0.94243604],
       [0.92728746],
       [0.4397073 ]], dtype=float32)

In [ ]:
sentiment_model.layers[0].set_weights

<bound method Layer.set_weights of <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f218d600e10>>

우리가 훈련한 모델을 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [ ]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.07      0.22      0.10       470
           1       0.96      0.85      0.90      9530

    accuracy                           0.82     10000
   macro avg       0.51      0.53      0.50     10000
weighted avg       0.91      0.82      0.86     10000



In [ ]:
from sklearn.metrics import confusion_matrix

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", Accuracy)
    print("Precision: ", Precision)
    print("Recall: ", Recall)
    print("Specificity: ", Specificity)
    print("F1-Score: ", F1_Score)

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print("Confusion Matrix")
print(confusion_matrix(y_true, np.round(preds,0)))
print("="*32)
print("Model Evaluation")
print(model_evaluation(y_true, np.round(preds,0)))

Confusion Matrix
[[ 102  368]
 [1455 8075]]
Model Evaluation
Accuracy:  0.8177
Precision:  0.9564135970626555
Recall:  0.8473242392444911
Specificity:  0.2170212765957447
F1-Score:  0.8985700773382296
None


In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_true, np.round(preds,0)))

0.5321727579201179


In [ ]:
import logging
tf.get_logger().setLevel(logging.ERROR)

# 실제 데이터로 실습하기

문장 하나 하나를 가지고 실제로 분류해보도록 하겠습니다.  

In [ ]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def sui_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    if predict_answer == 0:
      print("(자살 생각 X 확률 : %.2f)" % (1-predict_value))
    elif predict_answer == 1:
      print("(자살 생각 O 확률 : %.2f)" % predict_value)

In [ ]:
a = sentence_convert_data(train.document[0])

In [ ]:
sentiment_model.predict(a)

array([[0.21882394]], dtype=float32)

In [ ]:
predict_value = np.ravel(predict)
predict_value

array([0.21882394], dtype=float32)

In [ ]:
np.round(predict_value,0)

array([0.], dtype=float32)

In [ ]:
predict = sentiment_model.predict(a)
predict

array([[0.21882394]], dtype=float32)

In [ ]:
input_word_ids = a[0]        
input_masks= a[1]
input_segment =a[2]

In [ ]:
import numpy as np
import scipy

scipy.spatial.distance.cdist([input_word_ids, input_masks], [input_segment], metric='cosine')

ValueError: ignored

In [ ]:
sui_evaluation_predict(" 이게 말이 되냐 ")

(자살 생각 X 확률 : 0.80)


In [ ]:
for i in test.document[:30]:
  sui_evaluation_predict(i)

(자살 생각 X 확률 : 0.88)
(자살 생각 X 확률 : 0.75)
(자살 생각 O 확률 : 0.59)
(자살 생각 X 확률 : 0.97)
(자살 생각 O 확률 : 0.73)
(자살 생각 O 확률 : 0.82)
(자살 생각 X 확률 : 0.68)
(자살 생각 X 확률 : 0.85)
(자살 생각 X 확률 : 0.68)
(자살 생각 X 확률 : 0.97)
(자살 생각 X 확률 : 0.88)
(자살 생각 O 확률 : 0.53)
(자살 생각 X 확률 : 0.61)
(자살 생각 X 확률 : 0.97)
(자살 생각 X 확률 : 0.57)
(자살 생각 X 확률 : 0.98)
(자살 생각 X 확률 : 0.87)
(자살 생각 O 확률 : 0.66)
(자살 생각 O 확률 : 0.77)
(자살 생각 X 확률 : 0.97)
(자살 생각 X 확률 : 0.71)
(자살 생각 X 확률 : 0.89)
(자살 생각 X 확률 : 0.81)
(자살 생각 X 확률 : 0.73)
(자살 생각 X 확률 : 0.79)
(자살 생각 X 확률 : 0.94)
(자살 생각 O 확률 : 0.77)
(자살 생각 X 확률 : 0.73)
(자살 생각 X 확률 : 0.79)
(자살 생각 X 확률 : 0.84)
